# Huddinge Browser

This is a tool for browsing kmer enrichments in interactive two dimensional plots. This is work in progress so the user interface is a bit involved and the backend is quite fragile, but bare with me. I'm happy to take comments.

Here is a sample and tutorial for use of the system.

In [1]:
import sys
import logging as log
log.basicConfig(level=log.INFO,
                            format='%(asctime)s:%(funcName)s:%(levelname)s:%(message)s')
import numpy as np
import holoviews as hv
import numpy as np
import pandas as pd
hv.extension('bokeh',logo=False)

In [2]:
!mkdir -p /tmp/kpalin/
%cd /tmp/kpalin
## Temporary directory


/tmp/kpalin


In [3]:
import huddinge_tsne_browser.tsne_mapper as htm 
import huddinge_tsne_browser.huddinge_browser as hhb
import huddinge_tsne_browser.datashaderselect

Two main modules of the system are the `huddinge_tsne_browser.tsne_mapper` and `huddinge_tsne_browser.huddinge_browser`.  The tsne_mapper class reads the input files and possibly lays out the input kmers if they have not been laid out before.  Huddinge_browser class is more for interfacing the user.


The distribution comes with 8 mers laid out with TSNE approximating Huddinge distance. Software for calculating all pairs Huddinge distance (and producing appropriate output) is in branch `huddinge_pairs` of git repository `https://github.com/kpalin/MODER.git` and the computation can be done with command line `python huddinge_tsne_browser`.


## Download data

First you need to compute 8 mer counts for some selex [experiment](https://www.ebi.ac.uk/ena/data/view/PRJEB3289)

Larger set can be found from study [PRJEB9797](https://www.ebi.ac.uk/ena/data/view/PRJEB9797) which is related to [Yin, Yimeng, et al. "Impact of cytosine methylation on DNA binding specificities of human transcription factors." Science 356.6337 (2017)](https://www.ncbi.nlm.nih.gov/pubmed/28473536)

In [4]:
PROJECT_ID = "PRJEB9797"

In [5]:
read_data = pd.read_table("http://www.ebi.ac.uk/ena/data/warehouse/filereport?accession={}&result=read_run".format(PROJECT_ID))

In [6]:
import requests
from lxml import etree

In [7]:

project_xml = requests.get("https://www.ebi.ac.uk/ena/data/view/{}&display=xml".format(PROJECT_ID))
project_xml = etree.fromstring(project_xml.content)
sample_names = project_xml.xpath("//XREF_LINK[DB='ENA-SAMPLE']/ID")[0].text

In [8]:

sample_xml = requests.get("https://www.ebi.ac.uk/ena/data/view/{}&display=xml".format(sample_names))
sample_xml = etree.fromstring(sample_xml.content)

In [9]:
sample_titles = {}
for s in sample_xml.xpath("//SAMPLE"):
    p_id = s.find("IDENTIFIERS")
    p_id = p_id.find("PRIMARY_ID")
    title = s.find("TITLE")
    sample_titles[p_id.text] = title.text
    

In [10]:
s = pd.Series(sample_titles)
read_data = read_data.set_index("secondary_sample_accession")
read_data["sample_title"] = pd.Series(sample_titles)


In [11]:
read_data["experiment_type"] = read_data.sample_title.str.extract("^(.*) sample",expand=True)

In [12]:
read_data["experiment_type"].fillna("Missing").value_counts()

mHT-SELEX                                   2654
HT-SELEX                                    2654
bHT-SELEX                                   1464
High salt concentration mHT-SELEX            304
High salt concentration HT-SELEX             304
Klf4 ChIP-seq                                 12
Whole genome bisulfite sequencing (WGBS)      11
Oct4 ChIP-seq                                 11
ATAC-seq                                       6
n-Myc ChIP-seq                                 6
Rabbit IgG ChIP-seq                            4
Mouse IgG ChIP-seq                             3
Goat IgG ChIP-seq                              3
MAX ChIP-exo                                   2
HOXB13 ChIP-seq                                2
Rabbit IgG ChIP-exo                            2
CEBPB ChIP-exo                                 1
Name: experiment_type, dtype: int64

In [13]:
read_data["binder"] = read_data.sample_title.str.extract("sample (.*)$",expand=True)

In [14]:
s = read_data.experiment_type.str.contains("SELEX")
d = read_data.loc[s].sample_alias.str.extract("^(.*)_((FL|eDBD)(_Bis|_Nor)?)_([0-9]+)_",expand=True)
print("SELEX cycle observation counts:")
d[4].fillna(-1).astype(int).value_counts()


SELEX cycle observation counts:


3    2228
4    2228
2    1462
1    1462
Name: 4, dtype: int64

In [15]:
d[1].value_counts()

eDBD        4100
FL          1816
eDBD_Nor     462
eDBD_Bis     462
FL_Bis       270
FL_Nor       270
Name: 1, dtype: int64

In [16]:
pd.crosstab(read_data.loc[s].experiment_type,d[1])

1,FL,FL_Bis,FL_Nor,eDBD,eDBD_Bis,eDBD_Nor
experiment_type,,,,,,
HT-SELEX,908,0,0,1746,0,0
High salt concentration HT-SELEX,0,0,0,304,0,0
High salt concentration mHT-SELEX,0,0,0,304,0,0
bHT-SELEX,0,270,270,0,462,462
mHT-SELEX,908,0,0,1746,0,0


In [17]:
read_data["binder"] = None
read_data.loc[s,"binder"] = d[0]
read_data["binder_type"] = None
read_data.loc[s,"binder_type"] = d[1]


read_data["selex_cycle"] = -1
read_data.loc[s,"selex_cycle"] = d[4].fillna(-1).astype(int)

In [18]:
read_data.selex_cycle.value_counts()

 3    2228
 4    2228
 2    1462
 1    1462
-1      63
Name: selex_cycle, dtype: int64

In [19]:
f = "read_data_{}.tsv".format(PROJECT_ID)
import os.path
if os.path.exists(f):
    read_data = pd.read_table(f,sep="\t")
else:
    read_data.to_csv(f,sep="\t")

# Select reads to fetch

Using HOXB13 and HNF4A selex

In [20]:
s_data = read_data.loc[read_data.binder.str.contains("HOXB13|HNF4A").fillna(False)&(read_data.experiment_type=="HT-SELEX")]
s_data[["binder","binder_type","selex_cycle","experiment_type","read_count","sample_alias"]]

,binder,binder_type,selex_cycle,experiment_type,read_count,sample_alias
secondary_sample_accession,,,,,,
ERS831847,HNF4A,eDBD,1,HT-SELEX,297070,HNF4A_eDBD_1_KO_TACCTT40NCGA
ERS831849,HNF4A,eDBD,2,HT-SELEX,324120,HNF4A_eDBD_2_KO_TACCTT40NCGA
ERS831851,HNF4A,eDBD,3,HT-SELEX,373075,HNF4A_eDBD_3_KO_TACCTT40NCGA
ERS831853,HNF4A,eDBD,4,HT-SELEX,267281,HNF4A_eDBD_4_KO_TACCTT40NCGA
ERS831975,HOXB13,eDBD,1,HT-SELEX,201113,HOXB13_eDBD_1_KN_TCACTT40NTTG
ERS831977,HOXB13,eDBD,2,HT-SELEX,294798,HOXB13_eDBD_2_KN_TCACTT40NTTG
ERS831979,HOXB13,eDBD,3,HT-SELEX,249143,HOXB13_eDBD_3_KN_TCACTT40NTTG
ERS831981,HOXB13,eDBD,4,HT-SELEX,208431,HOXB13_eDBD_4_KN_TCACTT40NTTG


In [21]:
for i in s_data.fastq_ftp:
    !wget --no-clobber {i}

--2018-05-14 09:14:22--  http://ftp.sra.ebi.ac.uk/vol1/fastq/ERR100/006/ERR1003746/ERR1003746.fastq.gz
Resolving ftp.sra.ebi.ac.uk (ftp.sra.ebi.ac.uk)... 193.62.192.7
Connecting to ftp.sra.ebi.ac.uk (ftp.sra.ebi.ac.uk)|193.62.192.7|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12143487 (12M) [application/octet-stream]
Saving to: ‘ERR1003746.fastq.gz’

100%[======================================>] 12,143,487   709KB/s   in 17s    

2018-05-14 09:14:40 (704 KB/s) - ‘ERR1003746.fastq.gz’ saved [12143487/12143487]

--2018-05-14 09:14:40--  http://ftp.sra.ebi.ac.uk/vol1/fastq/ERR100/008/ERR1003748/ERR1003748.fastq.gz
Resolving ftp.sra.ebi.ac.uk (ftp.sra.ebi.ac.uk)... 193.62.192.7
Connecting to ftp.sra.ebi.ac.uk (ftp.sra.ebi.ac.uk)|193.62.192.7|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13293554 (13M) [application/octet-stream]
Saving to: ‘ERR1003748.fastq.gz’

100%[======================================>] 13,293,554   715KB/s   in 18s

In [22]:
print("Samples with null binders:")
read_data.loc[read_data.binder.isnull(),"sample_title"].value_counts()

Samples with null binders:


Klf4 ChIP-seq sample                               12
Oct4 ChIP-seq sample                               11
Whole genome bisulfite sequencing (WGBS) sample    11
n-Myc ChIP-seq sample                               6
ATAC-seq sample                                     6
Rabbit IgG ChIP-seq sample                          4
Mouse IgG ChIP-seq sample                           3
Goat IgG ChIP-seq sample                            3
Rabbit IgG ChIP-exo sample                          2
HOXB13 ChIP-seq sample                              2
MAX ChIP-exo sample                                 2
CEBPB ChIP-exo sample                               1
Name: sample_title, dtype: int64

## Calculate kmer counts

Then calculate 8mer counts for your data. Currently only jellyfish text output is good. (Also note jellyfish needs the `--disk` option for 8 and 7 mers)

In [23]:
%%bash  
K=8

for i in *.fastq.gz
do
    
    OUT=$(basename $i .fastq.gz).${K}mer_counts.jf
    #rm $OUT
    echo $OUT
    if [ ! -e ${OUT} ];
    then
        zcat $i | /usr/bin/time -v jellyfish count --canonical -o $OUT --text -m ${K} -s 1M --bf-size 1G -t 16 --disk /dev/stdin &
    fi
done
wait

ERR1003746.8mer_counts.jf
ERR1003748.8mer_counts.jf
ERR1003750.8mer_counts.jf
ERR1003752.8mer_counts.jf
ERR1003874.8mer_counts.jf
ERR1003876.8mer_counts.jf
ERR1003878.8mer_counts.jf
ERR1003880.8mer_counts.jf


	Command being timed: "jellyfish count --canonical -o ERR1003752.8mer_counts.jf --text -m 8 -s 1M --bf-size 1G -t 16 --disk /dev/stdin"
	User time (seconds): 6.98
	System time (seconds): 47.85
	Percent of CPU this job got: 383%
	Elapsed (wall clock) time (h:mm:ss or m:ss): 0:14.31
	Average shared text size (kbytes): 0
	Average unshared data size (kbytes): 0
	Average stack size (kbytes): 0
	Average total size (kbytes): 0
	Maximum resident set size (kbytes): 1234140
	Average resident set size (kbytes): 0
	Major (requiring I/O) page faults: 2
	Minor (reclaiming a frame) page faults: 361835
	Voluntary context switches: 7997
	Involuntary context switches: 5939
	Swaps: 0
	File system inputs: 632
	File system outputs: 840
	Socket messages sent: 0
	Socket messages received: 0
	Signals delivered: 0
	Page size (bytes): 4096
	Exit status: 0
	Command being timed: "jellyfish count --canonical -o ERR1003750.8mer_counts.jf --text -m 8 -s 1M --bf-size 1G -t 16 --disk /dev/stdin"
	User time (seconds): 

## Read kmer counts and select enriched ones


Calculating 8mer counts.

In [24]:
kmer_size=8
d={}
for _,s in s_data.iterrows():
    f = "{0.run_accession}.{1}mer_counts.jf".format(s,kmer_size)
    _,kmers = htm.read_jf(f)
    d[(s.binder,s.selex_cycle)]=kmers 
    
kmers = pd.DataFrame(d)


Normalize the kmer counts and return 
1. Normalized counts (count of a kmer divided by median count on that cycle)
2. Natural logarithm of fold change of normalized kmer count between each cycle
3. Mean of the log fold changes
4. z-score of the log fold change (mean divided by standard deviation)


In [25]:
def normalize_selex(kmers):
    """Normalize kmer counts. kmer.shape == (kmers,cycles).
    
    Return value is a tuple of 
    (median normalized counts, 
       log fold change for each cycle, 
       mean fold change per cycle,
       fold change z-score) """
    import pandas as pd 
    
    assert kmers.shape[0]> kmers.shape[1]
    kmers = kmers.sort_index(axis=1)+1

    # Median normalisation
    norm_cnt = kmers/kmers.median(axis=0)

    # log fold change per cycle
    ln_fold_change = np.log(norm_cnt).diff(axis=1)
    ln_fold_change = ln_fold_change.drop(ln_fold_change.columns[0],axis=1)

    # Mean fold change per cycle
    mean_fold = ln_fold_change.mean(axis=1)

    fold_z = mean_fold/ln_fold_change.std(axis=1)
    
    d = {}
    for i,c in enumerate(norm_cnt.columns):
        d[(c,"normalized")] = norm_cnt[c]
        if i>0:
            d[(c,"ln_fold")] = ln_fold_change[c]
            
    d["mean_ln_fold"] = mean_fold
    d["fold_z"] = fold_z
    return pd.DataFrame(d)
    d = {"normalized":norm_cnt,
         "ln_fold":ln_fold_change}
    return d#pd.DataFrame(d)
#         "mean_ln_fold":mean_fold,
#         "fold_z":fold_z}
    return d#pd.DataFrame(d)
    return norm_cnt,ln_fold_change,mean_fold,fold_z

In [26]:
data = dict()
for binder in ["HOXB13","HNF4A"]:
    data[binder] = normalize_selex(kmers[binder])
data = pd.concat(data)

Select kmers that enrich during selex with 99% confidence. i.e. enrichment z-score > 2.58

In [27]:
GAUSSIAN99 = 2.58
z = data["fold_z"].unstack().T
selected_kmers = z.loc[z.max(axis=1)>GAUSSIAN99]
selected_kmers_HNF4A = z.loc[z.HNF4A>GAUSSIAN99]
selected_kmers_HOXB13 = z.loc[z.HOXB13>GAUSSIAN99]

## Initialize layout

First you need to lay out your kmers. You can compute huddinge distances between all pairs of given kmers with accompanying `all_pairs_huddinge` command. Its output can be laid out with command line `huddinge_tsne_browser`command and the resulting should be given to `TsneMapper("kmer_layout.tsnet")`. TSNE layout of all 8mers takes more than an hour.  


For 8mers, the layout has already been done and the layout can be downloaded from [here](http://www.cs.helsinki.fi/u/kpalin/kmer8_iters4k.tsne.gz).


In [28]:
import os.path
if not os.path.exists("kmer8_iters4k.tsne"):
    !wget --no-clobber http://www.cs.helsinki.fi/u/kpalin/kmer8_iters4k.tsne.gz
    !gunzip  kmer8_iters4k.tsne.gz


--2018-05-14 09:16:51--  http://www.cs.helsinki.fi/u/kpalin/kmer8_iters4k.tsne.gz
Resolving www.cs.helsinki.fi (www.cs.helsinki.fi)... 128.214.166.78
Connecting to www.cs.helsinki.fi (www.cs.helsinki.fi)|128.214.166.78|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.cs.helsinki.fi/u/kpalin/kmer8_iters4k.tsne.gz [following]
--2018-05-14 09:16:51--  https://www.cs.helsinki.fi/u/kpalin/kmer8_iters4k.tsne.gz
Connecting to www.cs.helsinki.fi (www.cs.helsinki.fi)|128.214.166.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 169402557 (162M) [application/x-gzip]
Saving to: ‘kmer8_iters4k.tsne.gz’

100%[======================================>] 169,402,557 87.6MB/s   in 1.8s   

2018-05-14 09:16:53 (87.6 MB/s) - ‘kmer8_iters4k.tsne.gz’ saved [169402557/169402557]



In [29]:
import huddinge_tsne_browser.tsne_mapper as htm 
import huddinge_tsne_browser.huddinge_browser as hhb
import huddinge_tsne_browser.datashaderselect


In [30]:
try:
    from importlib import reload
except ImportError:
    pass

reload(huddinge_tsne_browser.datashaderselect)

reload(htm)


<module 'huddinge_tsne_browser.tsne_mapper' from '/home/kpalin/software/huddinge_tsne_browser/huddinge_tsne_browser/tsne_mapper.pyc'>

## Compute spectral embedding


The software uses sklearn [manifold](http://scikit-learn.org/stable/modules/manifold.html) learning to compute [spectral](http://scikit-learn.org/stable/modules/manifold.html#spectral-embedding), [MDS](http://scikit-learn.org/stable/modules/manifold.html#multi-dimensional-scaling-mds) or [tsne](http://scikit-learn.org/stable/modules/manifold.html#t-distributed-stochastic-neighbor-embedding-t-sne) embedding.


Here we use spectral embedding. MDS results in a ball and t-sne produces artificial clumps and is slow.



In [31]:
%pdb off
tsne = htm.TsneMapper("kmer8_iters4k.tsne",force_distances=True)

Automatic pdb calling has been turned OFF


2018-05-14 09:17:04,587:read_data:INFO:Read 65536 sequences.
2018-05-14 09:17:04,589:read_data:INFO:Setting embedding from input data
2018-05-14 09:17:04,606:read_data:INFO:Memory usage 403.754MB
2018-05-14 09:17:04,610:read_data:INFO:Reading distances.
2018-05-14 09:17:20,191:read_data:INFO:Memory usage 2451.73MB


In [32]:
selected_kmers_HOXB13.head()

,HNF4A,HOXB13
Sequence,,
ATAGACTA,-1.796374,2.882451
ACGAGAGT,0.776640,27.750740
CTCATAAA,0.311741,2.618992
GAGCAATA,-0.064896,3.745666
ACGCTCAT,2.288841,3.019603


In [33]:
#tsne.subset_sequences(list(selected_kmers.index))
tsne.subset_sequences(list(selected_kmers_HOXB13.index))
#tsne.subset_sequences(list(selected_kmers_HNF4A.index))

tsne.compute_spectral()

len(tsne)

2018-05-14 09:17:32,960:_get_matrix:INFO:Memory usage before matrix formatting 423.363MB
2018-05-14 09:17:32,962:_get_matrix:INFO:Reshaping..
2018-05-14 09:17:32,967:_get_matrix:INFO:sizeof(distances) = 0.737329MB
2018-05-14 09:17:32,970:_get_matrix:INFO:sizeof(M) = 5.90338MB
2018-05-14 09:17:32,985:_get_matrix:INFO:sizeof(idx) = 11.7973MB
2018-05-14 09:17:33,000:_get_matrix:INFO:Memory usage after matrix formatting 423.379MB
2018-05-14 09:17:33,130:_get_matrix:INFO:Memory usage after cleaning 423.379MB
/home/kpalin/software/huddinge_tsne_browser/huddinge_tsne_browser/tsne_mapper.py:291: RuntimeWarning: divide by zero encountered in divide
  adjacency[is_smallish] = 1.0 / self.matrix[is_smallish]
/home/kpalin/software/huddinge_tsne_browser/huddinge_tsne_browser/tsne_mapper.py:324: UserWarning: Disconnected kmers in adjacency graph
  warnings.warn("Disconnected kmers in adjacency graph")
/home/kpalin/miniconda2/lib/python2.7/site-packages/sklearn/manifold/spectral_embedding_.py:234: Use

1244

In [34]:
data.head()

fold_z  mean_ln_fold  (1, normalized)  (2, ln_fold)  \
      Sequence                                                          
HNF4A AAAAAAAA -0.610506     -0.197934         0.777344     -0.065223   
      ACGTACGA  4.997213      0.153770         0.968750      0.176539   
      GATCATTA -0.189814     -0.032095         0.644531      0.156369   
      GCCGAGCA  0.091850      0.003265         1.082031      0.011186   
      AACGTTGT  2.281009      0.169604         0.941406      0.140423   

                (2, normalized)  (3, ln_fold)  (3, normalized)  (4, ln_fold)  \
      Sequence                                                                 
HNF4A AAAAAAAA         0.728261     -0.567446         0.412903      0.038869   
      ACGTACGA         1.155797      0.166010         1.364516      0.118763   
      GATCATTA         0.753623     -0.170506         0.635484     -0.082150   
      GCCGAGCA         1.094203      0.034188         1.132258     -0.035578   
      AACGTTGT         1.083333      0.254123         1.396774      0.114266   

                (4, normalized)  
      Sequence                   
HNF4A AAAAAAAA         0.429268  
      ACGTACGA         1.536585  
      GATCATTA         0.585366  
      GCCGAGCA         1.092683  
      AACGTTGT         1.565854

Find maximally enriched sequences in local (in huddinge space) context.

In [35]:
huddinge_mat = pd.DataFrame(tsne.matrix,index=tsne.sequences.squeeze(),columns=tsne.sequences.squeeze())
def get_local_maxima(skmers,n=10,max_local_dist=1.0):
    candidates = skmers.sort_values(ascending=False).iteritems()
    #Find local maxima
    local_maxima  = [ next(candidates)[0] ]
    for kmer,v in candidates:
        neighbours = skmers[huddinge_mat.loc[kmer]<=max_local_dist]
        neighbours = neighbours.drop(kmer)
        if neighbours.max() < v:  # All neighbours are lower, hence we have local maxima
            local_maxima.append(kmer)
        else:
            log.info("Skipping non maximal locus {}".format(kmer))
        if len(local_maxima)>=n:
            break
    return local_maxima

In [36]:
loc_max = get_local_maxima(data.loc["HOXB13"].loc[selected_kmers_HOXB13.index,"mean_ln_fold"],
                           n=10)

2018-05-14 09:17:34,231:get_local_maxima:INFO:Skipping non maximal locus TCGTAAAA
2018-05-14 09:17:34,235:get_local_maxima:INFO:Skipping non maximal locus GCTCGTAA
2018-05-14 09:17:34,238:get_local_maxima:INFO:Skipping non maximal locus CCTCGTAA
2018-05-14 09:17:34,242:get_local_maxima:INFO:Skipping non maximal locus AGCTCGTA
2018-05-14 09:17:34,245:get_local_maxima:INFO:Skipping non maximal locus CGTAAAAC
2018-05-14 09:17:34,249:get_local_maxima:INFO:Skipping non maximal locus ACCTCGTA
2018-05-14 09:17:34,252:get_local_maxima:INFO:Skipping non maximal locus TCTCGTAA
2018-05-14 09:17:34,258:get_local_maxima:INFO:Skipping non maximal locus ATCTCGTA
2018-05-14 09:17:34,262:get_local_maxima:INFO:Skipping non maximal locus ATTTACGA
2018-05-14 09:17:34,265:get_local_maxima:INFO:Skipping non maximal locus GGCTCGTA
2018-05-14 09:17:34,269:get_local_maxima:INFO:Skipping non maximal locus CGCTCGTA
2018-05-14 09:17:34,272:get_local_maxima:INFO:Skipping non maximal locus ACTCGTAA
2018-05-14 09:17

Find optimal radial spacing for the local maxima (anchors). The r distance is the enrichment value.

In [37]:
def polar2cartesian(r,thetas):
    import numpy as np
    import pandas as pd
    import scipy.spatial.distance as ssd

    
    cartes = np.array([r*np.cos(thetas), r*np.sin(thetas)]).T
    if hasattr(thetas,"index"):
        cartes = pd.DataFrame(cartes,columns=["x","y"],index=thetas.index)
    return cartes


def circle_distances(r_thetas):

    import numpy as np
    import scipy.spatial.distance as ssd

    r,thetas = r_thetas[0],r_thetas[1:]
    cartes = np.array([r*np.cos(thetas), r*np.sin(thetas)]).T

    D = ssd.pdist(cartes,"euclidean")
    return D

def circle_map_anchors(anchors):

    
    
    import scipy.spatial.distance as ssd
    jitter = np.random.uniform(-0.3,0.3,size=(len(anchors),len(anchors)))
    np.fill_diagonal(jitter,0)
    jittered_dist = huddinge_mat.loc[anchors,anchors] + (jitter + jitter.T)/2.0
    jittered_D = ssd.squareform(jittered_dist)
    
    def ssq_distance_diff(r_thetas):
        return ( (circle_distances(r_thetas)-jittered_D)**2).sum()
    
    import scipy.optimize as so
    init_params = np.append( np.array([np.max(jittered_D)/2]), np.random.uniform(0,2*np.pi,size=len(anchors)))
    
    bounds = [(0.1,None)] + [(0.0,2.0*np.pi)]*len(anchors)
    
    #opt = so.minimize(ssq_distance_diff,x0=init_params,bounds = bounds)
    opt = so.basinhopping(ssq_distance_diff,x0=init_params,minimizer_kwargs = dict(bounds = bounds))
    
    print(opt.message)
    r = opt.x[0]
    thetas = pd.Series(opt.x[1:],index=anchors)
    D = pd.DataFrame(ssd.squareform(circle_distances(opt.x)),index=anchors,columns=anchors)
    _n = len(anchors)*(len(anchors)-1)/2
    return r,thetas,D,opt.fun/_n,((D-jittered_dist)**2).sum().sum()/2
    

In [38]:
%pdb off
ret = circle_map_anchors(loc_max)
ret[3:]

Automatic pdb calling has been turned OFF
['requested number of basinhopping iterations completed successfully']


(1.2971669054361803, 58.372510744628109)

In [44]:
ret[1]

CTCGTAAA    4.805243
GTTTACGA    4.169788
TACGAGCA    5.575925
ATTTTACG    3.646248
CAATAAAA    1.502974
GGTTTTAC    3.252167
ACGAGGTG    6.111474
ACGAGATA    0.435687
ACGAGGCA    0.051419
GTGTTTTA    2.830379
dtype: float64

In [45]:
enrichment_r=ret[1]


In [46]:
representatives = huddinge_mat.loc[enrichment_r.index]
representatives.index.name="representative"
representatives.columns.name="kmer"

representatives = representatives[representatives==representatives.min()].T.stack()
representatives.name="distance"

In [47]:
representatives.index.get_level_values("kmer").nunique()

1244

In [48]:
representatives.index.get_level_values("representative").nunique()

10

In [49]:
hv.HeatMap(
representatives.reset_index().groupby(["kmer","distance"]
                       ).size().reset_index(
                        ).rename(columns={0:"representatitives"}
                        ).groupby(["distance","representatitives"]).size().reset_index(
                        ).rename(columns={0:"kmers"}),
    kdims=["distance","representatitives"],vdims=["kmers"]
).opts(plot=dict(colorbar=True,width=500))#.bars(dimension=["representatitives"])#,"distance"])

:HeatMap   [distance,representatitives]   (kmers)

Above: most of the kmers are 2-4 edits away from their representative and there are multiple of them.

In [50]:
import matplotlib.pyplot as plt
fig,axs=plt.subplots(2,1)
representatives.groupby(level=["kmer"]).size().value_counts().sort_index().plot("bar",ax=axs[0])
#plt.title("Number of closest anchors")
#plt.ylabel("Number of kmers")
#plt.xlabel("Number of representatives")

representatives.groupby(level=["kmer"]).min().value_counts().sort_index().plot("bar",ax=axs[1])
axs[1].set_xlabel("Distance to representative")
axs[0].set_xlabel("Number of candidaterepresentatives")
axs[0].set_ylabel("kmers")
axs[1].set_ylabel("kmers")

Text(0,0.5,'kmers')

In [51]:
#enrichment_r = selected_kmers_HOXB13["HOXB13"].loc[ret[1].index]



# Position anchors
theta_angle = ret[1]
enrichment_r = data.loc["HOXB13"].loc[theta_angle.index,"mean_ln_fold"]
anchors_x = polar2cartesian(enrichment_r,theta_angle)
anchors_x["enrichment"] = enrichment_r



#Position others
single_rep = representatives.groupby("kmer").apply(lambda x:x.sample(1))
single_rep.index.names=["d"] + list(single_rep.index.names)[1:]
single_rep=single_rep.reset_index("d",drop=True).reset_index("representative")
single_rep["enrichment"] = data.loc["HOXB13"].loc[single_rep.index,"mean_ln_fold"].values
single_rep["theta"] = theta_angle[single_rep.representative].values


# Add angular jitter
jitter_span = 2*np.pi/100.0*single_rep.distance


single_rep[["x","y"]] = polar2cartesian(single_rep.enrichment,
                single_rep.theta + np.random.uniform(low=-jitter_span,high=jitter_span,size=len(jitter_span)) )

In [52]:
single_rep.head()

,representative,distance,enrichment,theta,x,y
kmer,,,,,,
AAAAAACG,CAATAAAA,3.0,0.216625,1.502974,-0.021307,0.215575
AAAAACAT,CAATAAAA,3.0,0.268421,1.502974,0.036972,0.265863
AAAAACGA,GTTTACGA,4.0,0.282510,4.169788,-0.179041,-0.218532
AAAAACGC,CAATAAAA,4.0,0.313212,1.502974,-0.000312,0.313212
AAAAACGG,CAATAAAA,4.0,0.281023,1.502974,0.005653,0.280967


In [53]:
x=single_rep.reset_index()

p = hv.Points(x,kdims=["x","y"],
          vdims=[ _x for _x in x.columns if _x not in ["x","y"]],
              extents=(-1,-1,1,1)
         ).opts(plot=dict(tools=["hover"],
                          width=600,height=500,
                          scaling_factor=13,size_index="enrichment",
                          show_grid=True,color_index="distance",colorbar=True,
                         )) * hv.Ellipse(0,0,1.0)
p.opts(plot=dict(aspect_weight=1.0,weight=1.0))

:Overlay
   .Points.I  :Points   [x,y]   (kmer,representative,distance,enrichment,theta)
   .Ellipse.I :Ellipse   [x,y]

In [54]:
selected_kmers_HOXB13["HOXB13"]
a = huddinge_mat.loc[loc_max].idxmin()
a.index.name=None
a=a.reset_index()

In [55]:


huddinge_mat.stack().loc[zip(a["index"],a[0])].value_counts().sort_index()

0.0     10
1.0     84
2.0    240
3.0    425
4.0    452
5.0     33
dtype: int64

In [56]:

def plot_polar(tf,theta_angle):
    "Arguments: Name of the binder tf and list of representative kmers (with angle positions)"
    %opts Points (cmap="inferno_r") [tools=['box_select', 'lasso_select'] color_index="mean_ln_fold"  scaling_factor=50 width=500 height=500]  { +axiswise -framewise }
    #%%opts Histogram (cmap="inferno_r") { +axiswise -framewise }
    
    from holoviews import streams
    
    
    

    # Position anchors
    enrichment_r = data.loc[tf].loc[theta_angle.index,"mean_ln_fold"]
    anchors_x = polar2cartesian(enrichment_r,theta_angle)
    anchors_x["enrichment"] = enrichment_r



    #Position others
    single_rep = representatives.groupby("kmer").apply(lambda x:x.sample(1))
    single_rep.index.names=["d"] + list(single_rep.index.names)[1:]
    single_rep=single_rep.reset_index("d",drop=True).reset_index("representative")
    single_rep["enrichment"] = data.loc[tf].loc[single_rep.index,"mean_ln_fold"].values
    single_rep["theta"] = theta_angle[single_rep.representative].values


    # Add angular jitter
    jitter_span = 2*np.pi/100.0*single_rep.distance


    single_rep[["x","y"]] = polar2cartesian(single_rep.enrichment,
                    single_rep.theta + np.random.uniform(low=-jitter_span,high=jitter_span,size=len(jitter_span)) )



    # Declare some points
    x=single_rep.reset_index()

    points = hv.Points(x,kdims=["x","y"],
          vdims=[ _x for _x in x.columns if _x not in ["x","y"]],
              extents=(-1,-1,1,1)
         ).opts(plot=dict(tools=["hover",'box_select', 'lasso_select'],
                          width=600,height=500,
                          scaling_factor=13,size_index="enrichment",
                          show_grid=True,color_index="distance",colorbar=True,

                         ),norm=dict(axiswise=True,framewise=False))
    #points.opts(plot=dict(aspect_weight=1.0,weight=1.0))
    
    
    
    #points = hv.Points(tsneD[tf].embedding.join(data.loc[tf]).reset_index(),
    #                   kdims=kdims,
    #                   vdims=['mean_ln_fold',"Sequence"]).relabel(tf)

    # Declare points as source of selection stream
    selection = streams.Selection1D(source=points)

    ENRICHMENT = "enrichment"
    # Write function that uses the selection indices to slice points and compute stats
    def selected_histogram(index):
        selected = points.iloc[index]
        if index:
            #label = str(selected.dframe().mean(axis=0))[:15]
            label = "Mean {} {}: {:.3g}".format(tf,ENRICHMENT,
                                             selected.dframe()[ENRICHMENT].mean())
            #label = 'Mean x, y: %.3f, %.3f' % tuple(selected.array().mean(axis=0))
        else:

            selected = points
            label = 'No selection'
        from holoviews.operation import histogram

        h = histogram(selected,dimension=ENRICHMENT,dynamic=False).relabel(label)#.opts(style=dict(color='red'))
        return h

    def selected_table(index):
        selected = points.iloc[index]
        if index:
            label = "Mean {} {}: {:.3g}".format(tf,ENRICHMENT,
                                             selected.dframe()[ENRICHMENT].mean())
        else:

            selected = points
            label = 'No selection'
        #from holoviews.operation import histogram
        #h = histogram(selected,dynamic=False).relabel(label)#.opts(style=dict(color='red'))
        return selected.table()
    
    def selected_matrix(index):
        import pandas as pd
        import holoviews as hv
        selected = points[[ENRICHMENT]].iloc[index]
        if not index:
            selected = points
        d= selected.data.kmer
        counts = pd.DataFrame(tuple(x) for x in d).apply(lambda x:x.value_counts(),axis=0)
        counts = counts.fillna(0)
        counts.columns = map(str,counts.columns)
        #return hv.Div(counts.astype(int).to_html(notebook=True)).opts(plot=dict(width=400,height=700))
        return hv.Table(counts)

    def show_logo(mat):
        import svgwrite as sw
        mat = (mat+1.0/(mat.shape[0]*mat.shape[1]))
        mat = mat/mat.sum()
        w = mat.shape[1]*15
        dr=sw.Drawing(size=("%dpt"%(w),"30pt"))
        #dr.add(dr.rect((0,0),(4100,1130),fill="green"))
        g = sw.container.Group()
        cmap = dict(zip("ACGT",["green","blue","orange","red"]))
        for c in mat.columns:

            xpos = 10*int(c)
            ypos = 10
            for i,base in enumerate(mat.index):
                yscale = mat.loc[base,c]

                t=g.add(dr.text(base,fill=cmap.get(base) ))
                t.translate(xpos,ypos)
                t.scale(sx=1,sy=yscale)
                ypos-=(10*yscale)
        dr.add(g)
        g.scale(sx=2,sy=4)
        return hv.Div(dr.tostring())#.opts(plot=dict(width=400,height=700))
        
    
    def selected_heatmap(index):
        import pandas as pd
        import holoviews as hv
        selected = points.iloc[index]
        if not index:
            selected = points
        d= selected.data.kmer
        counts = pd.DataFrame(tuple(x) for x in d).apply(lambda x:x.value_counts(),axis=0)
        counts = counts.fillna(0)
        try:
            return show_logo(counts)
        except (ImportError,AttributeError):
            counts = counts.stack().reset_index()
            counts.columns = map(str,counts.columns)

            t = hv.Table(counts,kdims=["level_1","level_0"])
            t = t.redim(level_1="Position",level_0="Base")


            return t.to.heatmap().opts(plot={"colorbar":True,"tools":["hover"],"invert_yaxis":True,
                                            "sizing_mode":"fixed",
                                             "width":400,"height":200})
        
    # Combine points and DynamicMap
    

    r=points.hist(dimension=ENRICHMENT)* hv.Ellipse(0,0,1.0) << hv.DynamicMap(selected_histogram, streams=[selection]) 
    r= r+hv.DynamicMap(selected_table, streams=[selection]) + \
        hv.DynamicMap(selected_matrix, streams=[selection]) #+ \
    #    hv.DynamicMap(selected_heatmap, streams=[selection])

    #r._selection = selection
    
    return r

In [59]:
%pdb
plot_polar("HOXB13",ret[1]).cols(1)#.hist(dimension="enrichment")

Automatic pdb calling has been turned ON


:Layout
   .AdjointLayout.I :AdjointLayout
      :DynamicMap   []
         :Histogram   [enrichment]   (enrichment_frequency)
      :Overlay
         .Points.I  :Points   [x,y]   (kmer,representative,distance,enrichment,theta)
         .Ellipse.I :Ellipse   [x,y]
      :Histogram   [enrichment]   (enrichment_frequency)
   .DynamicMap.I    :DynamicMap   []
      :Table   [x,y]   (kmer,representative,distance,enrichment,theta)
   .DynamicMap.II   :DynamicMap   []
      :Table   [0,1,2,3,4,5,6,7]

In [60]:
huddinge_mat.stack().value_counts().sort_index()

0.0      1244
1.0      6146
2.0     38864
3.0    159744
4.0    445446
5.0    667200
6.0    222110
7.0      6782
dtype: int64

In [61]:
huddinge_mat.loc[loc_max].min().value_counts().sort_index()

0.0     10
1.0     84
2.0    240
3.0    425
4.0    452
5.0     33
dtype: int64

Only 71 enriched kmers are within huddinge distance one from the top 10 local maxima.  It might work better if require locality to be within distance two.

In [62]:
huddinge_mat.loc[loc_max2].min().value_counts().sort_index()

NameError: name 'loc_max2' is not defined

> <ipython-input-62-9f46014eb185>(1)<module>()
----> 1 huddinge_mat.loc[loc_max2].min().value_counts().sort_index()

ipdb> 
ipdb> q


In [63]:
s = huddinge_mat.loc[kmer]
neighbours = skmers[s==1.0]
neighbours

NameError: name 'kmer' is not defined

> <ipython-input-63-ff0b4cbec87e>(1)<module>()
----> 1 s = huddinge_mat.loc[kmer]
      2 neighbours = skmers[s==1.0]
      3 neighbours

ipdb> q


In [82]:
kmer,neighbours.max(),v

('ATAAAAAG', 7.5712099725797435, 70.846999109211367)

Comp

In [64]:
%pdb off
tsneD =dict()
for tf in ("HOXB13","HNF4A"):
    tsneD[tf] = htm.TsneMapper("kmer8_iters4k.tsne",force_distances=True)

#tsne.subset_sequences(list(selected_kmers.index))
tsneD["HOXB13"].subset_sequences(list(selected_kmers_HOXB13.index))
tsneD["HNF4A"].subset_sequences(list(selected_kmers_HNF4A.index))
#tsne.subset_sequences(list(selected_kmers_HNF4A.index))
for tf in ("HOXB13","HNF4A"):
    tsneD[tf].compute_spectral()
    print(tf,len(tsneD[tf]))

Automatic pdb calling has been turned OFF


2018-05-14 09:28:56,175:read_data:INFO:Read 65536 sequences.
2018-05-14 09:28:56,176:read_data:INFO:Setting embedding from input data
2018-05-14 09:28:56,180:read_data:INFO:Memory usage 565.309MB
2018-05-14 09:28:56,182:read_data:INFO:Reading distances.
2018-05-14 09:28:58,995:read_data:INFO:Memory usage 2613.28MB
2018-05-14 09:28:59,211:read_data:INFO:Read 65536 sequences.
2018-05-14 09:28:59,213:read_data:INFO:Setting embedding from input data
2018-05-14 09:28:59,217:read_data:INFO:Memory usage 2615.21MB
2018-05-14 09:28:59,218:read_data:INFO:Reading distances.
2018-05-14 09:29:05,041:read_data:INFO:Memory usage 4663.18MB
2018-05-14 09:29:07,160:_get_matrix:INFO:Memory usage before matrix formatting 567.277MB
2018-05-14 09:29:07,162:_get_matrix:INFO:Reshaping..
2018-05-14 09:29:07,163:_get_matrix:INFO:sizeof(distances) = 0.737329MB
2018-05-14 09:29:07,168:_get_matrix:INFO:sizeof(M) = 5.90338MB
2018-05-14 09:29:07,185:_get_matrix:INFO:sizeof(idx) = 11.7973MB
2018-05-14 09:29:07,198:_g

('HOXB13', 1244)


2018-05-14 09:29:07,798:compute_spectral:INFO:Memory usage after embedding fit 608.461MB


('HNF4A', 1648)


In [65]:
#z = data.unstack(0)[[("fold_z","HNF4A")]]
#z.columns=["HNF4A_z"]
z = data.unstack(0)["mean_ln_fold"]
tsne.set_kmer_values(z)
tsne.embedding.head()

,spectral0,spectral1,HNF4A,HOXB13
Sequence,,,,
AAAAAACG,-0.202440,-0.217495,-0.152903,0.216625
AAAAACAT,-0.199490,-0.226319,-0.164899,0.268421
AAAAACGA,-0.219247,-0.221332,-0.126609,0.282510
AAAAACGC,-0.207638,-0.200888,-0.189539,0.313212
AAAAACGG,-0.212923,-0.224521,-0.079680,0.281023


## Browse

Create the browser module and display the browsing window. Browsing tools are selectable top right. The main display top left shows the kmers laid out colored according to the counts loaded above.  By clicking the main display you get table of kmers in the selected rectangle top right and more detailed figure, with point wise hover tool for counts at the bottom.  The coloring criterion of the main plot can be selected from the drop down menu.  These interactive features require jupyter running in the server.


In [66]:

reload(hhb)
%pdb off
br=hhb.HuddingBrowser(tsne)
p = br.holoview_plot()
#p

Automatic pdb calling has been turned OFF
('Diverging palette', (-0.58051956466764498, 0.58051956466764498), (-0.28551579205711652, 0.58051956466764498))


2018-05-14 09:29:13,216:__init__:INFO:Initialized DataShaderSelect


A pandas dataframe of selected kmers can be obtained from br.selected attribute.
The selected samples (all samples, all samples in zoom plot or samples selected in zoom plot) can be worked with in pandas:

In [67]:
#br._data.embedding.head()

In [68]:
%%opts Points (cmap="inferno_r") [color_index=2 size_index=3 scaling_factor=50 width=500 height=500]

points = hv.Points(br._data.embedding, kdims=["spectral0","spectral1"],vdims=['HOXB13'])
#points.hist()
#
#points.hist()

In [69]:
%%opts Points (cmap="inferno_r") [tools=['box_select', 'lasso_select'] color_index='HOXB13'  scaling_factor=50 width=500 height=500] 
#%%opts Histogram (cmap="inferno_r") { +axiswise -framewise }
from holoviews import streams
# Declare some points
points = hv.Points(br._data.embedding.reset_index(),kdims=["spectral0","spectral1"],vdims=['HOXB13',"Sequence"])

# Declare points as source of selection stream
selection = streams.Selection1D(source=points)

# Write function that uses the selection indices to slice points and compute stats
def selected_info(index):
    selected = points.iloc[index]
    if index:
        #label = str(selected.dframe().mean(axis=0))[:15]
        label = "Mean {}: {:.3g}".format(selected.vdims[0],selected.dframe()[str(selected.vdims[0])].mean())
        #label = 'Mean x, y: %.3f, %.3f' % tuple(selected.array().mean(axis=0))
    else:
        
        selected = points
        label = 'No selection'
    from holoviews.operation import histogram
    return histogram(selected,dynamic=False).relabel(label)#.opts(style=dict(color='red'))

# Combine points and DynamicMap
r=points.hist()<< hv.DynamicMap(selected_info, streams=[selection]) 
#r

In [70]:

def plot_tsne(tf,kdims=["spectral0","spectral1"]):
    %opts Points (cmap="inferno_r") [tools=['box_select', 'lasso_select'] color_index="mean_ln_fold"  scaling_factor=50 width=500 height=500]  { +axiswise -framewise }
    #%%opts Histogram (cmap="inferno_r") { +axiswise -framewise }
    
    from holoviews import streams
    # Declare some points
    points = hv.Points(tsneD[tf].embedding.join(data.loc[tf]).reset_index(),
                       kdims=kdims,
                       vdims=['mean_ln_fold',"Sequence"]).relabel(tf)

    # Declare points as source of selection stream
    selection = streams.Selection1D(source=points)

    # Write function that uses the selection indices to slice points and compute stats
    def selected_info(index):
        selected = points.iloc[index]
        if index:
            #label = str(selected.dframe().mean(axis=0))[:15]
            label = "Mean {}: {:.3g}".format(selected.vdims[0],selected.dframe()[str(selected.vdims[0])].mean())
            #label = 'Mean x, y: %.3f, %.3f' % tuple(selected.array().mean(axis=0))
        else:

            selected = points
            label = 'No selection'
        from holoviews.operation import histogram
        return histogram(selected,dynamic=False).relabel(label)#.opts(style=dict(color='red'))

    # Combine points and DynamicMap
    r=points.hist()<< hv.DynamicMap(selected_info, streams=[selection]) 
    r._selection = selection
    return r

In [71]:

def plot_tsne(tf,kdims=["spectral0","spectral1"]):
    %opts Points (cmap="inferno_r") [tools=['box_select', 'lasso_select'] color_index="mean_ln_fold"  scaling_factor=50 width=500 height=500]  { +axiswise -framewise }
    #%%opts Histogram (cmap="inferno_r") { +axiswise -framewise }
    
    from holoviews import streams
    # Declare some points
    points = hv.Points(tsneD[tf].embedding.join(data.loc[tf]).reset_index(),
                       kdims=kdims,
                       vdims=['mean_ln_fold',"Sequence"]).relabel(tf)

    # Declare points as source of selection stream
    selection = streams.Selection1D(source=points)

    # Write function that uses the selection indices to slice points and compute stats
    def selected_histogram(index):
        selected = points.iloc[index]
        if index:
            #label = str(selected.dframe().mean(axis=0))[:15]
            label = "Mean {}: {:.3g}".format(selected.vdims[0],selected.dframe()[str(selected.vdims[0])].mean())
            #label = 'Mean x, y: %.3f, %.3f' % tuple(selected.array().mean(axis=0))
        else:

            selected = points
            label = 'No selection'
        from holoviews.operation import histogram
        h = histogram(selected,dynamic=False).relabel(label)#.opts(style=dict(color='red'))
        return h

    def selected_table(index):
        selected = points.iloc[index]
        if index:
            #label = str(selected.dframe().mean(axis=0))[:15]
            label = "Mean {}: {:.3g}".format(selected.vdims[0],selected.dframe()[str(selected.vdims[0])].mean())
            #label = 'Mean x, y: %.3f, %.3f' % tuple(selected.array().mean(axis=0))
        else:

            selected = points
            label = 'No selection'
        #from holoviews.operation import histogram
        #h = histogram(selected,dynamic=False).relabel(label)#.opts(style=dict(color='red'))
        return selected.table()
    
    def selected_matrix(index):
        import pandas as pd
        import holoviews as hv
        selected = points.iloc[index]
        if not index:
            selected = points
        d= selected.data.Sequence
        counts = pd.DataFrame(tuple(x) for x in d).apply(lambda x:x.value_counts(),axis=0)
        counts = counts.fillna(0)
        counts.columns = map(str,counts.columns)
        #return hv.Div(counts.astype(int).to_html(notebook=True)).opts(plot=dict(width=400,height=700))
        return hv.Table(counts)

    def show_logo(mat):
        import svgwrite as sw
        mat = (mat+1.0/(mat.shape[0]*mat.shape[1]))
        mat = mat/mat.sum()
        w = mat.shape[1]*15
        dr=sw.Drawing(size=("%dpt"%(w),"30pt"))
        #dr.add(dr.rect((0,0),(4100,1130),fill="green"))
        g = sw.container.Group()
        cmap = dict(zip("ACGT",["green","blue","orange","red"]))
        for c in mat.columns:

            xpos = 10*int(c)
            ypos = 10
            for i,base in enumerate(mat.index):
                yscale = mat.loc[base,c]

                t=g.add(dr.text(base,fill=cmap.get(base) ))
                t.translate(xpos,ypos)
                t.scale(sx=1,sy=yscale)
                ypos-=(10*yscale)
        dr.add(g)
        g.scale(sx=2,sy=4)
        return hv.Div(dr.tostring())#.opts(plot=dict(width=400,height=700))
        
    
    def selected_heatmap(index):
        import pandas as pd
        import holoviews as hv
        selected = points.iloc[index]
        if not index:
            selected = points
        d= selected.data.Sequence
        counts = pd.DataFrame(tuple(x) for x in d).apply(lambda x:x.value_counts(),axis=0)
        counts = counts.fillna(0)
        try:
            return show_logo(counts)
        except (ImportError,AttributeError):
            counts = counts.stack().reset_index()
            counts.columns = map(str,counts.columns)

            t = hv.Table(counts,kdims=["level_1","level_0"])
            t = t.redim(level_1="Position",level_0="Base")


            return t.to.heatmap().opts(plot={"colorbar":True,"tools":["hover"],"invert_yaxis":True,
                                            "sizing_mode":"fixed",
                                             "width":400,"height":200})
        
    # Combine points and DynamicMap
    

    r=points.hist()<< hv.DynamicMap(selected_histogram, streams=[selection]) 
    r= r+hv.DynamicMap(selected_table, streams=[selection]) + \
        hv.DynamicMap(selected_matrix, streams=[selection]) + \
        hv.DynamicMap(selected_heatmap, streams=[selection])

    r._selection = selection
    
    return r

In [72]:
r_HNF4A=plot_tsne("HNF4A")
r_HNF4A.cols(2)

:Layout
   .AdjointLayout.HNF4A :AdjointLayout
      :DynamicMap   []
         :Histogram   [mean_ln_fold]   (mean_ln_fold_frequency)
      :Points   [spectral0,spectral1]   (mean_ln_fold,Sequence)
      :Histogram   [mean_ln_fold]   (mean_ln_fold_frequency)
   .DynamicMap.I        :DynamicMap   []
      :Table   [spectral0,spectral1]   (mean_ln_fold,Sequence)
   .DynamicMap.II       :DynamicMap   []
      :Table   [0,1,2,3,4,5,6,7]
   .DynamicMap.III      :DynamicMap   []
      :Div

In [73]:
#tsneD["HOXB13"].embedding
#r= hv.HoloMap({tf:plot_tsne(tf) for tf in tsneD.keys()})
r=plot_tsne("HOXB13").cols(2)
r#+r.data["main"].iloc[r._selection.index].table()

:Layout
   .AdjointLayout.HOXB13 :AdjointLayout
      :DynamicMap   []
         :Histogram   [mean_ln_fold]   (mean_ln_fold_frequency)
      :Points   [spectral0,spectral1]   (mean_ln_fold,Sequence)
      :Histogram   [mean_ln_fold]   (mean_ln_fold_frequency)
   .DynamicMap.I         :DynamicMap   []
      :Table   [spectral0,spectral1]   (mean_ln_fold,Sequence)
   .DynamicMap.II        :DynamicMap   []
      :Table   [0,1,2,3,4,5,6,7]
   .DynamicMap.III       :DynamicMap   []
      :Div

In [329]:
#r.AdjointLayout.HOXB13.data["main"].iloc[r._selection.index].table()
x=r.DynamicMap.II.dframe()
x.to_dict()
#print(r)

{'0': {'A': 59.0, 'C': 39.0, 'G': 0.0, 'T': 4.0},
 '1': {'A': 9.0, 'C': 54.0, 'G': 39.0, 'T': 0.0},
 '2': {'A': 39.0, 'C': 9.0, 'G': 54.0, 'T': 0.0},
 '3': {'A': 54.0, 'C': 0.0, 'G': 48.0, 'T': 0.0},
 '4': {'A': 15.0, 'C': 19.0, 'G': 68.0, 'T': 0.0},
 '5': {'A': 26, 'C': 21, 'G': 28, 'T': 27},
 '6': {'A': 30, 'C': 25, 'G': 25, 'T': 22},
 '7': {'A': 40, 'C': 32, 'G': 19, 'T': 11}}

In [307]:
hv.help(hv.Div)

Div

Online example: http://holoviews.org/reference/elements/bokeh/Div.html

-------------
Style Options
-------------

	<No style options available>

------------
Plot Options
------------

The plot options are the parameters of the plotting class:

Parameters of 'DivPlot'

Parameters changed from their default values are marked in red.
Soft bound values are marked in cyan.
C/V= Constant/Variable, RO/RW = ReadOnly/ReadWrite, AN=Allow None

Name                            Value                    Type         Bounds     Mode  

apply_extents                    True                  Boolean        (0, 1)     V RW  
apply_ranges                     True                  Boolean        (0, 1)     V RW  
bgcolor                          None               ClassSelector              V RW AN 
finalize_hooks                    []                   HookList     (0, None)    V RW  
fontsize                         None                 Parameter                V RW AN 
height                    

## Custom values in plot

You can replace the kmer counts with numbers you count yourself.

In [5]:
import numpy as np
np.concatenate([[1,2,3],[3,4,5]])

array([1, 2, 3, 3, 4, 5])

In [ ]:
# Median normalisation
x = tsne.embedding[tsne.data_dims]
norm_cnt = x/x.median(axis=0)


# log fold change per cycle
ln_fold_change = np.log(norm_cnt[ [x for x in norm_cnt.columns if x.startswith("HNF4A")] ]).diff(axis=1).drop("HNF4A_1",axis=1)
ln_fold_change2 =  np.log(norm_cnt[ [x for x in norm_cnt.columns if x.startswith("HOXB13")] ]).diff(axis=1).drop("HOXB13_1",axis=1)


annot = tsne.embedding.join(norm_cnt,rsuffix="_norm").join(pd.concat([ln_fold_change,ln_fold_change2],axis=1),
                                                           rsuffix="_lnfold")
annot["HNF4A_MeanFold"] = ln_fold_change.mean(axis=1)
annot["HOXB13_MeanFold"] = ln_fold_change2.mean(axis=1)


In [ ]:
annot.head().T

In [ ]:
# Pseudocount for kmer count estimates
p_cnt = (tsne.embedding[tsne.data_dims]+1.0)

# Scale to mean count 1.0
norm_cnt = p_cnt/(p_cnt*4**-8).sum()

# log fold change per cycle
ln_fold_change = np.log(norm_cnt).diff(axis=1).drop("HNF4A_1",axis=1)

# Mean fold change weighted by number of reads
w_mean_ln_fold_change = (p_cnt*ln_fold_change ).sum(axis=1)/p_cnt.sum(axis=1)

annot = tsne.embedding.join(norm_cnt,rsuffix="_norm").join(ln_fold_change,rsuffix="_lnfold")
annot["MeanFold"] = w_mean_ln_fold_change


In [ ]:
import huddinge_tsne_browser.tsne_mapper as htm 
import huddinge_tsne_browser.huddinge_browser as hhb
import huddinge_tsne_browser.datashaderselect
reload(huddinge_tsne_browser.datashaderselect)
reload(hhb)

reload(htm)
%pdb off

In [ ]:
annot.sort_values("HOXB13_MeanFold",ascending=False).head(20).T

In [36]:
from holoviews.operation.datashader import datashade
import datashader as ds
datashade(hv.Points(x),aggregator=ds.count(0)) 

NameError: name 'x' is not defined

In [39]:
ds.transfer_functions.shade??

In [ ]:

annot["HOXB13_t"] = annot["HOXB13_MeanFold"]/ annot[["HOXB13_{:d}_lnfold".format(x+2) for x in range(3)]].std(axis=1)
annot["HNF4A_t"] = annot["HNF4A_MeanFold"]/ annot[["HNF4A_{:d}_lnfold".format(x+2) for x in range(3)]].std(axis=1)

In [ ]:
reload(ds)
reload(holoviews.operation.datashader)
import holoviews.operation.datashader as hod
reload(hod)
import datashader as ds
annot["HOXB13_t"] = annot["HOXB13_MeanFold"]/ annot[["HOXB13_{:d}_lnfold".format(x+2) for x in range(3)]].std(axis=1)
annot["HNF4A_t"] = annot["HNF4A_MeanFold"]/ annot[["HNF4A_{:d}_lnfold".format(x+2) for x in range(3)]].std(axis=1)

x = annot[["HNF4A_t","HOXB13_t"]]#.rename(columns={"HOXB13_t":"B","HNF4A_t":"A"})
x.loc[x.T.abs().max()<100]
#x = annot[["HNF4A_MeanFold","HOXB13_MeanFold"]].copy()#.rename(columns={"HOXB13_MeanFold":"B","HNF4A_MeanFold":"A"})

#datashade(hv.Points(x),aggregator=ds.count(0))
p = hv.Points(x[["HNF4A_t","HOXB13_t"]])
hod.datashade(p) 

In [ ]:
p

In [ ]:
from holoviews.operation.datashader import datashade
import datashader as ds
datashade(hv.Points(annot[["HNF4A_t","HOXB13_t"]]),aggregator=ds.count(0))

In [ ]:
x=annot[["HNF4A_t","HOXB13_t"]]
x.isnull().any()

In [ ]:
import copy
tsne_fold = copy.deepcopy(tsne)
#tsne_fold = htm.TsneMapper("kmer8_iters4k.tsne")
tsne_fold.set_kmer_values(annot[[x for x in annot.columns if x not in ["tsne0","tsne1"]]])
#tsne_fold.set_kmer_values(annot[["MeanFold"]])

br_fold = hhb.HuddingBrowser(tsne_fold)

br_fold.holoview_plot()



### Note on coloring

The datashaded main plot is normalized with histogram normalization to maximize the dynamic range of the color palette. This might result in surprisng color scales.

The colormap for non-negative (or non-positive) data is viridis and for data with both positive and negative values,  it is RdYlBu (Red Yellow Blue) but that is not very informative because of the histogram equalisation/normalisation.